In [ ]:
import os
import shutil
import cv2
import numpy as np

In [ ]:
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)

In [ ]:
# Use the code below to find each nucleus center of the nucleus image and crop each channel based on those centers

In [ ]:
mkdir('./angii_drug_image/crop_images')
for ch in range(1, 5):
    mkdir(f'./angii_drug_image/crop_images/ch{ch}_crop')

In [ ]:
mkdir('./iso_drug_image/crop_images')
for ch in range(1, 5):
    mkdir(f'./iso_drug_image/crop_images/ch{ch}_crop')

In [ ]:
# Define a function to find the center coordinates of the nucleus, returning a list of coordinates (tuples) of each center
def find_centers(img):
    gradX = cv2.Sobel(img, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(img, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)

    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)

    blurred = cv2.blur(gradient,(9, 9))
    (_, thresh) = cv2.threshold(blurred, 247, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)

    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cs = sorted(cnts, key=cv2.contourArea, reverse=True)[:int(len(cnts)*0.66)]

    # The center of each nucleus in the image
    centers = []
    for c in cs:
        rect = cv2.minAreaRect(c)
        center = rect[0]
        centers.append(center)

    return centers

In [ ]:
# angii group image cropping
# The nucleus image is the CH1 channel, if it is other channels, it needs to be modified
filenames_ch1 = os.listdir('./angii_drug_image/whole_images/ch1/')             # filenames_ch1 represents an image of the nuclear channel
for fn_ch1 in filenames_ch1:
    img_nuclei = cv2.imread(f'./angii_drug_image/whole_images/ch1/{fn_ch1}', cv2.IMREAD_UNCHANGED)
    centers = find_centers(img=img_nuclei)

    fn_ch2 = f"{fn_ch1[:22]}"+"2"+f"{fn_ch1[23:]}"
    fn_ch3 = f"{fn_ch1[:22]}"+"3"+f"{fn_ch1[23:]}"
    fn_ch4 = f"{fn_ch1[:22]}"+"4"+f"{fn_ch1[23:]}"

    img2 = cv2.imread(os.path.join('./angii_drug_image/whole_images/ch2/',fn_ch2), cv2.IMREAD_UNCHANGED)
    img3 = cv2.imread(os.path.join('./angii_drug_image/whole_images/ch3/',fn_ch3), cv2.IMREAD_UNCHANGED)
    img4 = cv2.imread(os.path.join('./angii_drug_image/whole_images/ch4/',fn_ch4), cv2.IMREAD_UNCHANGED)

    idx = 0
    for center in centers:
        x = int(center[0])
        y = int(center[1])
        if y-150 >= 0 and x-150 >= 0 and y+150 <= 1080 and x+150 <= 1080:
            img1_crop = img_nuclei[(y-150):(y+150), (x-150):(x+150)]
            img2_crop = img2[(y-150):(y+150), (x-150):(x+150)]
            img3_crop = img3[(y-150):(y+150), (x-150):(x+150)]
            img4_crop = img4[(y-150):(y+150), (x-150):(x+150)]

            idx +=1

            cv2.imwrite(f"./angii_drug_image/crop_images/ch1_crop/{fn_ch1[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img1_crop)
            cv2.imwrite(f"./angii_drug_image/crop_images/ch2_crop/{fn_ch2[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img2_crop)
            cv2.imwrite(f"./angii_drug_image/crop_images/ch3_crop/{fn_ch3[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img3_crop)
            cv2.imwrite(f"./angii_drug_image/crop_images/ch4_crop/{fn_ch4[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img4_crop)

In [ ]:
# iso group image cropping
# The nucleus image is the CH1 channel, if it is other channels, it needs to be modified
filenames_ch1 = os.listdir('./iso_drug_image/whole_images/ch1/')               # filenames_ch1 represents an image of the nuclear channel
for fn_ch1 in filenames_ch1:
    img_nuclei = cv2.imread(f'./iso_drug_image/whole_images/ch1/{fn_ch1}', cv2.IMREAD_UNCHANGED)
    centers = find_centers(img=img_nuclei)

    fn_ch2 = f"{fn_ch1[:22]}"+"2"+f"{fn_ch1[23:]}"
    fn_ch3 = f"{fn_ch1[:22]}"+"3"+f"{fn_ch1[23:]}"
    fn_ch4 = f"{fn_ch1[:22]}"+"4"+f"{fn_ch1[23:]}"

    img2 = cv2.imread(os.path.join('./iso_drug_image/whole_images/ch2/',fn_ch2), cv2.IMREAD_UNCHANGED)
    img3 = cv2.imread(os.path.join('./iso_drug_image/whole_images/ch3/',fn_ch3), cv2.IMREAD_UNCHANGED)
    img4 = cv2.imread(os.path.join('./iso_drug_image/whole_images/ch4/',fn_ch4), cv2.IMREAD_UNCHANGED)

    idx = 0
    for center in centers:
        x = int(center[0])
        y = int(center[1])
        if y-150 >= 0 and x-150 >= 0 and y+150 <= 1080 and x+150 <= 1080:
            img1_crop = img_nuclei[(y-150):(y+150), (x-150):(x+150)]
            img2_crop = img2[(y-150):(y+150), (x-150):(x+150)]
            img3_crop = img3[(y-150):(y+150), (x-150):(x+150)]
            img4_crop = img4[(y-150):(y+150), (x-150):(x+150)]

            idx +=1

            cv2.imwrite(f"./iso_drug_image/crop_images/ch1_crop/{fn_ch1[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img1_crop)
            cv2.imwrite(f"./iso_drug_image/crop_images/ch2_crop/{fn_ch2[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img2_crop)
            cv2.imwrite(f"./iso_drug_image/crop_images/ch3_crop/{fn_ch3[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img3_crop)
            cv2.imwrite(f"./iso_drug_image/crop_images/ch4_crop/{fn_ch4[:-14]}"+f"crop{'%02d'%idx}"+".tiff", img4_crop)